I-A TAD Conjunto Disjunto

In [15]:
import numpy as np

def init_cd(n: int):
    return np.full((1,n), -1, dtype=int)

print(init_cd(5))


def union(rep_1: int, rep_2: int, p_cd: np.ndarray):
    if p_cd[rep_1] < p_cd[rep_2]:
        p_cd[rep_2] = rep_1
        print(p_cd) #tests
        return rep_1
    elif p_cd[rep_1] > p_cd[rep_2]:
        p_cd[rep_1] = rep_2
        print(p_cd) # tests
        return rep_2
    else:
        p_cd[rep_1] = rep_2
        p_cd[rep_1] -= 1
        print(p_cd) #tests
        return rep_2

test = np.array = [-4, 0, 5, 1, 0, 4, -3, 6, 7]
#print(union(0, 6, test))

def find(ind: int, p_cd: np.ndarray):
    root = ind
    while p_cd[root] >= 0:
        root  = p_cd[root]

    while p_cd[ind] >= 0:
        a = p_cd[ind]
        p_cd[ind] = root
        ind = a
    print(p_cd)
    return root
    

	
print(test)
print(find(2, test))



[[-1 -1 -1 -1 -1]]
[-4, 0, 5, 1, 0, 4, -3, 6, 7]
[-4, 0, 0, 1, 0, 0, -3, 6, 7]
0


II-A Algoritmo de Kruskall

In [ ]:
from queue import PriorityQueue

def create_pq(n: int, l_g: list):
    pq = PriorityQueue()

    for u, v, w in l_g:
        pq.put((w, (u,v)))
    

def kruskal(n: int, l_g: list):
    pq = create_pq(n, l_g)
    ds = init_cd(n)
    l_t = list

    while not pq.empty:
        _, (u, v) = pq.get()

        x = find(u, ds)
        y = find(v, ds)

        if x != y:
            l_t.append((u,v))
            union(u, v, ds)
    
    for i in range(0, len(ds)-1):
        

    return tuple[n, l_t]
